In [ ]:
from math import sqrt
from numpy import concatenate
import tensorflow as tf
from matplotlib import pyplot
from pandas import read_csv
from tensorflow import keras 
from pandas import DataFrame
from pandas import concat
from tensorflow.keras.optimizers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense,Dropout

from keras.layers import LSTM
import numpy as np 
import pickle 
import pandas as pd 
from sklearn.linear_model import LinearRegression


In [ ]:
dataset = read_csv('/content/station.csv')
#dataset=dataset.drop(['day','month','mintemp','Bias'],axis=1)
values = dataset.values
values.shape

# dataset.head(30)

(882, 3)

In [ ]:
scaler = pickle.load(open('/content/scaler_model', 'rb'))


In [ ]:
scaled = scaler.transform(values)

In [ ]:
scaled

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
# specify the number of lag hours
n_days = 7 
n_features =3
# frame as supervised learning
reframed = series_to_supervised(scaled, n_days, 1) 
# reframed
print(reframed.head())

In [ ]:
values = reframed.values 
values
n_train_days = 8251
train = values[:n_train_days, :] 

val = values[n_train_days:1028+n_train_days, :]  

test=values[1028+n_train_days:,:]
# test

In [ ]:
n_obs = n_days* n_features
print(n_obs)

21


In [ ]:
_,month= test[:, :n_obs], test[:, -n_features+2]   

In [ ]:
train_X_mlp, train_y = train[:, :n_obs], train[:, -n_features]
# # print(train_X[-1,:])
# # print(train_y[-1])
# val_X, val_y = val[:, :n_obs], val[:, -n_features]
test_X_mlp, test_y = test[:, :n_obs], test[:, -n_features]  
# print(test_X[0,:])
# print(test_y[0])
# print(val_X)

In [ ]:

loaded_model = pickle.load(open("/content/mlp_u90_adam_612", 'rb'))

In [ ]:
ypred_mlp = loaded_model.predict(test_X_mlp) 


In [ ]:

fcp1=np.repeat(ypred_mlp,13,axis=1)
inv_yhat=scaler.inverse_transform(fcp1)
inv_yhat = inv_yhat[:,0]

# print(inv_yhat)
test_y =test_y.reshape((len(test_y), 1))
fcp2=np.repeat(test_y,13,axis=1)
inv_y = scaler.inverse_transform(fcp2) 
inv_y = inv_y[:,0]  

test_rmse = sqrt(mean_squared_error(inv_y, inv_yhat))  
print(test_rmse)

0.981775566905665


In [ ]:

train_X_GRU= train_X_mlp.reshape((train_X_mlp.shape[0], n_days, n_features))
# val_X = val_X.reshape((val_X.shape[0],n_days,n_features))
test_X_GRU = test_X_mlp.reshape((test_X_mlp.shape[0], n_days, n_features))



# print(val_X.shape,train_X.shape,test_X.shape)


In [ ]:
loaded_model = pickle.load(open("/content/GRU_30_adam_mae_max", 'rb'))

In [ ]:
ypred_gru= loaded_model.predict(test_X_GRU) 
# print(ypred_gru.shape) 
fcp1=np.repeat(ypred_gru,13,axis=1)
inv_yhat=scaler.inverse_transform(fcp1)
inv_yhat = inv_yhat[:,0]

# print(inv_yhat)
test_y =test_y.reshape((len(test_y), 1))
fcp2=np.repeat(test_y,13,axis=1)
inv_y = scaler.inverse_transform(fcp2) 
inv_y = inv_y[:,0]  

test_rmse = sqrt(mean_squared_error(inv_y, inv_yhat))  
print(test_rmse)

0.9795522002064


In [ ]:
loaded_model = pickle.load(open("/content/lstm_7_adam_msl_with_month", 'rb'))

In [ ]:
ypred_lstm= loaded_model.predict(test_X_GRU)

In [ ]:

fcp1=np.repeat(ypred_lstm,13,axis=1)
inv_yhat=scaler.inverse_transform(fcp1)
inv_yhat = inv_yhat[:,0]

# print(inv_yhat)
test_y =test_y.reshape((len(test_y), 1))
fcp2=np.repeat(test_y,13,axis=1)
inv_y = scaler.inverse_transform(fcp2) 
inv_y = inv_y[:,0]  

test_rmse = sqrt(mean_squared_error(inv_y, inv_yhat)) 
print(test_rmse)


0.94740845900911


In [ ]:
preds = np.stack([ypred_mlp.reshape(1,-1)[0], ypred_gru.reshape(1,-1)[0],ypred_lstm.reshape(1,-1)[0]], axis=1)
preds 


In [ ]:
# stacker = LinearRegression()
# stacker.fit(X=preds, y=train_y) 
# # with open('Ensembled_model_for_max','wb') as f: 
# #   pickle.dump(stacker,f) 


In [ ]:
# stack_pred = stacker.predict(preds) 
ensembled_model = pickle.load(open("/content/Ensembled_model_for_max", 'rb'))

In [ ]:
ensembled_pred =  ensembled_model.predict(preds) 

In [ ]:
# stacker.score(preds,train_y) 

In [ ]:
stack_pred = ensembled_pred.reshape(132,1)

In [ ]:

fcp1=np.repeat(stack_pred,3,axis=1)
inv_yhat=scaler.inverse_transform(fcp1)
inv_yhat = inv_yhat[:,0]

# print(inv_yhat)
test_y =test_y.reshape((len(test_y), 1))
fcp2=np.repeat(test_y,3,axis=1)
inv_y = scaler.inverse_transform(fcp2) 
inv_y = inv_y[:,0]  

test_rmse = sqrt(mean_squared_error(inv_y, inv_yhat))

In [ ]:
test_rmse

0.9960899196980016